# <center>**Trabajo Práctico Número 2**</center>
### <center>**Regresión del valor de valor medio de casas en distritos de California**</center>
#### <center>Introducción a la Inteligencia Artificial - CEIA & DPLN, Fiuba</center>

<center><img width="35%" src="https://lh5.googleusercontent.com/cPAoMij8-mRhQEFe6cf2R1bncilBk29i0DaUS7EkqUcLb4IdcbjCUaBXcU9k-T59JJDuOkvsB_PDdKa0F3Q54_sAdIi_4tNK5oHYmT20OLV64mFLoiq2G2L-9ihkDdgJSg=w1280"/></center>

_________

**Integrantes del grupo**
- Argento, Lucas
- Espínola, Carla
- Gambarte, Antonella
- Putrino, Daniela
- Silvera, Ricardo
---------